# 🧠 Global Beer & Beverage Analytics



## Capa Bronze – Ingesta de datos crudos

In [1]:
# Configuración inicial
import os
import requests
import polars as pl
from datetime import datetime

# Crear carpeta bronze si no existe
os.makedirs("../data/bronze", exist_ok=True)

print("✅ Entorno cargado correctamente.")
print("Versión de Polars:", pl.__version__)


✅ Entorno cargado correctamente.
Versión de Polars: 1.34.0


In [2]:

os.makedirs("../data/bronze", exist_ok=True)

DATASETS = {
    "alcohol_total_percapita": "https://ourworldindata.org/grapher/total-alcohol-consumption-per-capita-litres-of-pure-alcohol.csv",
    "beer_percapita": "https://ourworldindata.org/grapher/beer-consumption-per-person.csv",
    "spirits_percapita": "https://ourworldindata.org/grapher/spirits-consumption-per-person.csv",
}

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
saved = []

for name, url in DATASETS.items():
    print(f"↓ Descargando {name} ...")
    r = requests.get(url, timeout=60)
    if r.status_code == 200 and r.content:
        out_path = f"../data/bronze/{name}_{ts}.csv"
        with open(out_path, "wb") as f:
            f.write(r.content)
        saved.append(out_path)
        print(f"  ✅ Guardado: {out_path}")
    else:
        print(f"  ⚠️ Error {r.status_code} al descargar {name} ({url})")

print("\nResumen Bronze:")
for p in saved:
    print("  •", p)


↓ Descargando alcohol_total_percapita ...
  ✅ Guardado: ../data/bronze/alcohol_total_percapita_20251027_190435.csv
↓ Descargando beer_percapita ...
  ✅ Guardado: ../data/bronze/beer_percapita_20251027_190435.csv
↓ Descargando spirits_percapita ...
  ✅ Guardado: ../data/bronze/spirits_percapita_20251027_190435.csv

Resumen Bronze:
  • ../data/bronze/alcohol_total_percapita_20251027_190435.csv
  • ../data/bronze/beer_percapita_20251027_190435.csv
  • ../data/bronze/spirits_percapita_20251027_190435.csv


In [3]:
# === NUEVO: Descargar PIB per cápita (World Bank / OWID) ===
import requests
from datetime import datetime
import os

os.makedirs("../data/bronze", exist_ok=True)

url_gdp = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"
r = requests.get(url_gdp)
if r.status_code == 200:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    gdp_path = f"../data/bronze/gdp_per_capita_{ts}.csv"
    with open(gdp_path, "wb") as f:
        f.write(r.content)
    print(f"✅ PIB guardado en: {gdp_path}")
else:
    print("⚠️ Error al descargar PIB")


✅ PIB guardado en: ../data/bronze/gdp_per_capita_20251027_190439.csv


In [4]:
# === BRONZE: Descargar metadata de países (región + income) desde la API del World Bank ===
import os, json, requests
from datetime import datetime

os.makedirs("../data/bronze", exist_ok=True)

# Paginamos por si superamos el default; 300 cubre todos los países/territorios
url = "https://api.worldbank.org/v2/country?format=json&per_page=300"
r = requests.get(url, timeout=60)
if r.status_code != 200:
    raise RuntimeError(f"Error {r.status_code} al llamar la API del World Bank")

payload = r.json()  # payload[1] es la lista de países
countries = payload[1] if isinstance(payload, list) and len(payload) > 1 else []

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
bronze_json = f"../data/bronze/wb_countries_{ts}.json"
with open(bronze_json, "w", encoding="utf-8") as f:
    json.dump(countries, f, ensure_ascii=False)

print(f"✅ Metadata de países guardada (JSON): {bronze_json}")

# tomamos los campos de country, iso3Code, region y incomeLevel 
# directamente de la fuente oficial (campos publicados por el World Bank)

✅ Metadata de países guardada (JSON): ../data/bronze/wb_countries_20251027_190440.json


# 🥈 Capa Silver – Limpieza y Normalización de Datos

En esta etapa transformamos los archivos crudos (Bronze) en datasets limpios, 
consistentes y auditables.

**Objetivos:**
1. Estandarizar nombres y tipos de columnas.  
2. Eliminar nulos y duplicados.  
3. Corregir valores negativos de consumo (errores históricos).  
4. Registrar métricas de calidad (porcentaje de limpieza y correcciones).  
5. Consolidar un único dataset `alcohol_consumption_all.parquet` para la capa Gold.


In [5]:
import polars as pl
import glob, os
from datetime import datetime


In [6]:
def read_and_clean(file_path: str, beverage: str) -> tuple[pl.DataFrame, dict]:
    """
    Lee un CSV desde Bronze, limpia y estandariza las columnas para la capa Silver.
    Retorna:
      - DataFrame limpio (Polars)
      - Métricas de calidad (diccionario)
    """
    raw_df = pl.read_csv(file_path)
    last_col = raw_df.columns[-1]
    total_raw = raw_df.height

    # Cast y limpieza base
    df = (
        raw_df.rename({
            "Entity": "country",
            "Code": "iso_code",
            "Year": "year",
            last_col: "litres_per_capita"
        })
        .with_columns([
            pl.col("year").cast(pl.Int32),
            pl.col("litres_per_capita").cast(pl.Float64)
        ])
    )

    # Métricas previas
    nulls = df.filter(pl.col("litres_per_capita").is_null()).height
    negatives = df.filter(pl.col("litres_per_capita") < 0).height

    # Limpieza final
    df_clean = (
        df.filter(pl.col("litres_per_capita").is_not_null())
          .with_columns(
              pl.when(pl.col("litres_per_capita") < 0)
                .then(0)
                .otherwise(pl.col("litres_per_capita"))
                .alias("litres_per_capita")
          )
          .unique()
          .with_columns(pl.lit(beverage).alias("beverage_type"))
    )

    cleaned_rows = df_clean.height
    quality_pct = round((cleaned_rows / total_raw) * 100, 2) if total_raw > 0 else 0

    metrics = {
        "dataset_name": os.path.basename(file_path),
        "beverage_type": beverage,
        "total_raw": total_raw,
        "cleaned_rows": cleaned_rows,
        "null_removed": nulls,
        "negative_fixed": negatives,
        "quality_pct": quality_pct,
        "run_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return df_clean, metrics


## 📥 Limpieza de datasets individuales (beer, spirits, total)
Cargamos los archivos CSV desde la capa Bronze y aplicamos la función `read_and_clean`
para estandarizarlos y generar métricas de calidad.


In [7]:
bronze_files = glob.glob("../data/bronze/*.csv")
logs = []

beer_df, beer_log = read_and_clean([f for f in bronze_files if "beer" in f][0], "beer")
spirits_df, spirits_log = read_and_clean([f for f in bronze_files if "spirits" in f][0], "spirits")
total_df, total_log = read_and_clean([f for f in bronze_files if "alcohol_total" in f][0], "total")

logs.extend([beer_log, spirits_log, total_log])

print("✅ Limpieza individual completada.")


✅ Limpieza individual completada.


## 🧩 Consolidación del dataset Silver unificado

Concatenamos los tres datasets limpios en un único archivo Parquet
`alcohol_consumption_all.parquet`, listo para análisis o enriquecimiento (PIB y regiones).


In [8]:
df_silver_all = pl.concat([beer_df, spirits_df, total_df], how="vertical")
df_silver_all.write_parquet("../data/silver/alcohol_consumption_all.parquet")

print(f"✅ Dataset Silver unificado guardado: {df_silver_all.height} filas totales")


✅ Dataset Silver unificado guardado: 24772 filas totales


## 🧾 Registro de Control de Calidad

Creamos o actualizamos el archivo `data_quality_log.parquet`, 
donde almacenamos las métricas de limpieza de cada ejecución.


In [9]:
log_df = pl.DataFrame(logs)
log_path = "../data/silver/data_quality_log.parquet"

if os.path.exists(log_path):
    existing_log = pl.read_parquet(log_path)
    log_df = pl.concat([existing_log, log_df], how="vertical")

log_df.write_parquet(log_path)
print("✅ Log de calidad actualizado correctamente.")
log_df


✅ Log de calidad actualizado correctamente.


dataset_name,beverage_type,total_raw,cleaned_rows,null_removed,negative_fixed,quality_pct,run_date
str,str,i64,i64,i64,i64,f64,str
"""bronze\beer_percapita_20251025…","""beer""",10304,10304,0,0,100.0,"""2025-10-26 19:47:42"""
"""bronze\spirits_percapita_20251…","""spirits""",10304,10304,0,6,100.0,"""2025-10-26 19:47:42"""
"""bronze\alcohol_total_percapita…","""total""",4164,4164,0,0,100.0,"""2025-10-26 19:47:42"""
"""beer_percapita_20251025_203406…","""beer""",10304,10304,0,0,100.0,"""2025-10-26 20:19:50"""
"""spirits_percapita_20251025_203…","""spirits""",10304,10304,0,6,100.0,"""2025-10-26 20:19:50"""
"""alcohol_total_percapita_202510…","""total""",4164,4164,0,0,100.0,"""2025-10-26 20:19:50"""
"""beer_percapita_20251025_203406…","""beer""",10304,10304,0,0,100.0,"""2025-10-27 19:04:40"""
"""spirits_percapita_20251025_203…","""spirits""",10304,10304,0,6,100.0,"""2025-10-27 19:04:40"""
"""alcohol_total_percapita_202510…","""total""",4164,4164,0,0,100.0,"""2025-10-27 19:04:40"""


## 🧠 Verificación rápida de consistencia

Validamos que:
- No haya valores negativos.
- Las columnas tengan los tipos esperados.


In [10]:
print("Filas con valores negativos:",
      df_silver_all.filter(pl.col("litres_per_capita") < 0).height)

print("\nTipos de columnas:")
print(df_silver_all.schema)


Filas con valores negativos: 0

Tipos de columnas:
Schema({'country': String, 'iso_code': String, 'year': Int32, 'litres_per_capita': Float64, 'beverage_type': String})


# 🥈 Capa Silver (Parte 2) – Enriquecimiento con PIB y Regiones del World Bank

En esta etapa añadimos contexto socioeconómico a los datos limpios de consumo:

**Fuentes:**
- PIB per cápita (World Bank / OWID)
- Clasificación regional e ingresos (World Bank API)

**Objetivos:**
1. Descargar, limpiar y normalizar los datasets externos.  
2. Unirlos al dataset limpio `alcohol_consumption_all.parquet` usando claves estándar (ISO3, país y año).  
3. Generar un dataset enriquecido `alcohol_consumption_enriched.parquet` listo para análisis avanzados.


In [11]:
import polars as pl
import requests, json, os, glob
from datetime import datetime


## 💰 Descarga y limpieza del PIB per cápita (World Bank / OWID)

Usamos el dataset oficial de GDP de World Bank (via GitHub datasets).
Se estandarizan nombres y tipos, y se guarda en formato Parquet.


In [12]:
os.makedirs("../data/bronze", exist_ok=True)

url_gdp = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"
r = requests.get(url_gdp, timeout=60)
if r.status_code == 200:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    gdp_path = f"../data/bronze/gdp_per_capita_{ts}.csv"
    with open(gdp_path, "wb") as f:
        f.write(r.content)
    print(f"✅ PIB descargado en: {gdp_path}")
else:
    raise RuntimeError("⚠️ Error al descargar PIB")

# Limpieza
gdp_df = (
    pl.read_csv(gdp_path)
    .rename({
        "Country Name": "country",
        "Country Code": "iso_code",
        "Year": "year",
        "Value": "gdp_per_capita_usd"
    })
    .with_columns([
        pl.col("year").cast(pl.Int32),
        pl.col("gdp_per_capita_usd").cast(pl.Float64)
    ])
    .filter(pl.col("gdp_per_capita_usd").is_not_null())
    .unique()
)

gdp_df.write_parquet("../data/silver/gdp_per_capita.parquet")
print("✅ Dataset PIB limpio guardado en: /data/silver/gdp_per_capita.parquet")
gdp_df.head(5)


✅ PIB descargado en: ../data/bronze/gdp_per_capita_20251027_190441.csv
✅ Dataset PIB limpio guardado en: /data/silver/gdp_per_capita.parquet


country,iso_code,year,gdp_per_capita_usd
str,str,i32,f64
"""Latin America & Caribbean (exc…","""LAC""",2014,5.4573e12
"""South Asia (IDA & IBRD)""","""TSA""",1978,1.7380e11
"""Liechtenstein""","""LIE""",1976,2.7249e8
"""Equatorial Guinea""","""GNQ""",2001,1.4611e9
"""Venezuela, RB""","""VEN""",1972,1.3978e10


## 🌍 Descarga y limpieza del dataset de regiones e ingresos (World Bank API)

Usamos el endpoint oficial del World Bank para obtener información de:
- Región (e.g. "Latin America & Caribbean")
- Income Group (e.g. "High income")
- Lending Type
- Capital City

Filtramos los registros con valor "Aggregates" (agregados globales sin datos válidos).


In [13]:
# Descargar JSON desde el API oficial del World Bank
url_regions = "https://api.worldbank.org/v2/country?format=json&per_page=300"
r = requests.get(url_regions, timeout=60)
if r.status_code != 200:
    raise RuntimeError(f"Error {r.status_code} al llamar la API del World Bank")

payload = r.json()
countries = payload[1] if isinstance(payload, list) and len(payload) > 1 else []

# Guardar copia en Bronze
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
bronze_json = f"../data/bronze/wb_countries_{ts}.json"
with open(bronze_json, "w", encoding="utf-8") as f:
    json.dump(countries, f, ensure_ascii=False)
print(f"✅ Metadata de países guardada: {bronze_json}")

# Limpieza y normalización
records = []
for c in countries:
    region_val = (c.get("region") or {}).get("value")
    income_val = (c.get("incomeLevel") or {}).get("value")
    lend_val   = (c.get("lendingType") or {}).get("value")

    if region_val == "Aggregates" or income_val == "Aggregates":
        continue

    records.append({
        "country": c.get("name"),
        "iso_code": c.get("id"),
        "region": region_val,
        "income_group": income_val,
        "lending_type": lend_val,
        "capital_city": c.get("capitalCity")
    })

regions_df = (
    pl.DataFrame(records)
      .with_columns([
          pl.col("country").cast(pl.Utf8),
          pl.col("iso_code").cast(pl.Utf8),
          pl.col("region").cast(pl.Utf8),
          pl.col("income_group").cast(pl.Utf8),
          pl.col("lending_type").cast(pl.Utf8),
          pl.col("capital_city").cast(pl.Utf8)
      ])
      .unique(subset=["iso_code"])
)

regions_df.write_parquet("../data/silver/country_regions.parquet")
print("✅ Dataset regiones guardado en: /data/silver/country_regions.parquet")
regions_df.head(10)


✅ Metadata de países guardada: ../data/bronze/wb_countries_20251027_190442.json
✅ Dataset regiones guardado en: /data/silver/country_regions.parquet


country,iso_code,region,income_group,lending_type,capital_city
str,str,str,str,str,str
"""Tunisia""","""TUN""","""Middle East, North Africa, Afg…","""Lower middle income""","""IBRD""","""Tunis"""
"""Cyprus""","""CYP""","""Europe & Central Asia""","""High income""","""Not classified""","""Nicosia"""
"""Isle of Man""","""IMN""","""Europe & Central Asia""","""High income""","""Not classified""","""Douglas"""
"""Korea, Rep.""","""KOR""","""East Asia & Pacific""","""High income""","""Not classified""","""Seoul"""
"""Argentina""","""ARG""","""Latin America & Caribbean ""","""Upper middle income""","""IBRD""","""Buenos Aires"""
"""Turkiye""","""TUR""","""Europe & Central Asia""","""Upper middle income""","""IBRD""","""Ankara"""
"""Tajikistan""","""TJK""","""Europe & Central Asia""","""Lower middle income""","""IDA""","""Dushanbe"""
"""Greenland""","""GRL""","""Europe & Central Asia""","""High income""","""Not classified""","""Nuuk"""
"""Somalia, Fed. Rep.""","""SOM""","""Sub-Saharan Africa ""","""Low income""","""IDA""","""Mogadishu"""


## 🧩 Unión de datasets: Consumo + PIB + Regiones

Combinamos las tres fuentes usando claves estándar:
- `country + year` para el PIB  
- `iso_code` para la región e income group

Guardamos el dataset final enriquecido en `/data/silver/alcohol_consumption_enriched.parquet`


In [14]:
df_cons = pl.read_parquet("../data/silver/alcohol_consumption_all.parquet")
df_gdp  = pl.read_parquet("../data/silver/gdp_per_capita.parquet")
df_reg  = pl.read_parquet("../data/silver/country_regions.parquet")

df_temp = df_cons.join(df_gdp, on=["country", "year"], how="left")
df_enriched = df_temp.join(df_reg, on="iso_code", how="left")

df_enriched.write_parquet("../data/silver/alcohol_consumption_enriched.parquet")
print(f"✅ Dataset enriquecido guardado: {df_enriched.height} filas")
df_enriched.select(["country", "iso_code", "region", "income_group"]).head(10)


✅ Dataset enriquecido guardado: 24772 filas


country,iso_code,region,income_group
str,str,str,str
"""Jamaica""","""JAM""","""Latin America & Caribbean ""","""Upper middle income"""
"""Kenya""","""KEN""","""Sub-Saharan Africa ""","""Lower middle income"""
"""Central African Republic""","""CAF""","""Sub-Saharan Africa ""","""Low income"""
"""El Salvador""","""SLV""","""Latin America & Caribbean ""","""Upper middle income"""
"""Yemen""","""YEM""","""Middle East, North Africa, Afg…","""Low income"""
"""Poland""","""POL""","""Europe & Central Asia""","""High income"""
"""Guyana""","""GUY""","""Latin America & Caribbean ""","""High income"""
"""Honduras""","""HND""","""Latin America & Caribbean ""","""Lower middle income"""
"""Mauritius""","""MUS""","""Sub-Saharan Africa ""","""Upper middle income"""


## 🧪 Validación de integridad del dataset enriquecido
Verificamos que los campos `region` e `income_group` se hayan poblado correctamente.


In [15]:
df_enriched.select([
    pl.col("region").null_count().alias("nulos_region"),
    pl.col("income_group").null_count().alias("nulos_income"),
    pl.col("country").n_unique().alias("num_paises"),
    pl.col("region").n_unique().alias("num_regiones"),
    pl.col("income_group").n_unique().alias("num_grupos_ingresos")
])


nulos_region,nulos_income,num_paises,num_regiones,num_grupos_ingresos
u32,u32,u32,u32,u32
397,397,202,8,6


# 📦 Resultado de la Capa Silver (completa)

Estructura final de carpeta `/data/silver/`:

/data/silver/
 - ├── alcohol_consumption_all.parquet        ← Silver limpio consolidado
 - ├── data_quality_log.parquet               ← Log de auditoría de calidad
 - ├── gdp_per_capita.parquet                 ← PIB por país y año
 - ├── country_regions.parquet                ← Regiones e ingresos (World Bank)
 - └── alcohol_consumption_enriched.parquet   ← Silver enriquecido listo para Gold

✅ Este es el dataset final que se usará en la Capa Gold para análisis, KPIs y visualizaciones.


# 🥇 Capa Gold – Análisis y Visualización Interactiva

En esta capa analizamos y visualizamos los datos enriquecidos para extraer información clave.

**Objetivos:**
1. Crear métricas globales y socioeconómicas.  
2. Generar gráficos de tendencias, rankings y correlaciones.  
3. Exportar resultados listos para Power BI y dashboards web.  

**Dataset base:**  
`/data/silver/alcohol_consumption_enriched.parquet`


In [16]:
import polars as pl
import plotly.express as px
import os

# Cargar dataset enriquecido
df = pl.read_parquet("../data/silver/alcohol_consumption_enriched.parquet")

print("✅ Dataset cargado con éxito.")
print("Filas:", df.height)
print("Columnas:", df.columns)


✅ Dataset cargado con éxito.
Filas: 24772
Columnas: ['country', 'iso_code', 'year', 'litres_per_capita', 'beverage_type', 'iso_code_right', 'gdp_per_capita_usd', 'country_right', 'region', 'income_group', 'lending_type', 'capital_city']


## 📈 Tendencia global por tipo de bebida (1960–2020)

Analizamos cómo ha evolucionado el consumo promedio global de alcohol per cápita
según el tipo de bebida.


In [17]:
trend_df = (
    df.group_by(["year", "beverage_type"])
      .agg(pl.mean("litres_per_capita").alias("avg_litres_per_capita"))
      .sort(["year", "beverage_type"])
)

fig = px.line(
    trend_df.to_pandas(),
    x="year",
    y="avg_litres_per_capita",
    color="beverage_type",
    title="Consumo promedio global de alcohol por tipo de bebida (1960–2020)",
    labels={
        "year": "Año",
        "avg_litres_per_capita": "Litros per cápita (alcohol puro)",
        "beverage_type": "Tipo de bebida"
    },
    markers=True
)
fig.update_layout(template="plotly_white", hovermode="x unified")
fig.show()

## 🌍 Consumo promedio por región

Comparamos el consumo promedio por región del mundo, mostrando patrones geográficos
a lo largo del tiempo.


In [18]:
region_df = (
    df.filter(pl.col("region").is_not_null())
      .group_by(["region", "year"])
      .agg(pl.mean("litres_per_capita").alias("avg_litres"))
      .sort(["region", "year"])
)

fig = px.line(
    region_df.to_pandas(),
    x="year",
    y="avg_litres",
    color="region",
    title="Consumo promedio de alcohol per cápita por región (1960–2020)",
    labels={"year": "Año", "avg_litres": "Litros per cápita"},
)
fig.update_layout(template="plotly_white", hovermode="x unified")
fig.show()


## 💰 Consumo promedio por grupo de ingresos

Mostramos la evolución del consumo según el nivel económico del país
(usando las categorías del Banco Mundial).


In [19]:
 
income_df = (
    df.filter(pl.col("income_group").is_not_null())
      .group_by(["income_group", "year"])
      .agg(pl.mean("litres_per_capita").alias("avg_litres"))
      .sort(["income_group", "year"])
)

fig = px.line(
    income_df.to_pandas(),
    x="year",
    y="avg_litres",
    color="income_group",
    title="Consumo de alcohol por grupo de ingresos (1960–2020)",
    labels={
        "year": "Año",
        "avg_litres": "Litros per cápita",
        "income_group": "Nivel de ingreso"
    },
    markers=True
)
fig.update_layout(template="plotly_white", hovermode="x unified")
fig.show()


## 🏆 Top 10 países con mayor consumo per cápita (último año disponible)


In [20]:
latest_year = df["year"].max()
ranking_df = (
    df.filter(pl.col("year") == latest_year)
      .group_by("country")
      .agg(pl.mean("litres_per_capita").alias("avg_litres"))
      .sort("avg_litres", descending=True)
      .head(10)
)

fig = px.bar(
    ranking_df.to_pandas(),
    x="avg_litres",
    y="country",
    orientation="h",
    title=f"Top 10 países con mayor consumo per cápita ({latest_year})",
    labels={"avg_litres": "Litros per cápita", "country": "País"},
)
fig.update_layout(template="plotly_white")
fig.show()


## ⚖️ Relación entre PIB per cápita y consumo de alcohol

Visualizamos cómo evoluciona la relación entre riqueza y consumo de alcohol 
a lo largo de los años (animación estilo Hans Rosling).


In [27]:
import polars as pl
import pandas as pd
import plotly.express as px

# Base limpia para el scatter
base = (
    df.filter(
        pl.col("gdp_per_capita_usd").is_not_null() &
        pl.col("litres_per_capita").is_not_null()
    )
    .with_columns(
        pl.when(pl.col("litres_per_capita") < 0).then(0).otherwise(pl.col("litres_per_capita")).alias("litres_per_capita"),
        pl.col("year").cast(pl.Int32)
    )
)

# 1) Encontrar un año que tenga TODAS las categorías de income_group
grp_per_year = (
    base.group_by("year")
        .agg(pl.col("income_group").n_unique().alias("cats"))
        .sort("year")
)

full_years = grp_per_year.filter(pl.col("cats") >= 5)["year"].to_list()
# fallback: si por alguna razón no hay 5, usa el primero con el máximo disponible
if not full_years:
    max_cats = grp_per_year["cats"].max()
    full_years = grp_per_year.filter(pl.col("cats") == max_cats)["year"].to_list()

first_year = int(full_years[0])

# 2) Orden de frames: arrancamos por el año "completo"
years_sorted = sorted(base["year"].unique().to_list())
years_sorted = [first_year] + [y for y in years_sorted if y != first_year]

# 3) Pasar a pandas y castear year a CATEGORÍA ORDENADA
df_scatter = (
    base.sort(["income_group", "year"])
        .to_pandas()
)
df_scatter["year"] = pd.Categorical(df_scatter["year"].astype(str),
                                    categories=[str(y) for y in years_sorted],
                                    ordered=True)

# 4) (Opcional) rangos fijos para más estabilidad de la animación
rx = [df_scatter["gdp_per_capita_usd"].min()*0.9, df_scatter["gdp_per_capita_usd"].max()*1.1]
ry = [0, df_scatter["litres_per_capita"].max()*1.1]

fig = px.scatter(
    df_scatter,
    x="gdp_per_capita_usd",
    y="litres_per_capita",
    color="income_group",
    size="litres_per_capita",
    animation_frame="year",
    animation_group="country",              # 💡 clave para animaciones consistentes
    hover_name="country",
    title="Relación entre PIB per cápita y consumo de alcohol (1960–2020)",
    labels={"gdp_per_capita_usd": "PIB per cápita (USD)",
            "litres_per_capita": "Consumo (litros per cápita)"},
    log_x=True,
    size_max=30,
    category_orders={"year": [str(y) for y in years_sorted]}
    # , range_x=rx, range_y=ry             # ← descomenta si quieres fijar rangos
)

fig.update_layout(template="plotly_white", legend_title_text="Grupo de ingreso",
                  xaxis_title="PIB per cápita (escala log)",
                  yaxis_title="Consumo (litros per cápita)")
fig.show()


## 💾 Exportación de resultados Gold
Guardamos las tablas analíticas (Parquet y CSV) para Power BI o visualización externa.


In [24]:
os.makedirs("../data/gold", exist_ok=True)

trend_df.write_parquet("../data/gold/global_trend_per_beverage.parquet")
region_df.write_parquet("../data/gold/region_trends.parquet")
income_df.write_parquet("../data/gold/income_trends.parquet")
ranking_df.write_parquet("../data/gold/top10_countries.parquet")

print("✅ Tablas Gold exportadas correctamente.")

✅ Tablas Gold exportadas correctamente.


## 🌐 Exportación de visualizaciones interactivas a HTML

Exportamos los gráficos en formato HTML para compartirlos o integrarlos en Power BI / web.


In [1]:
plots_path = "../data/gold/plots"
os.makedirs(plots_path, exist_ok=True)

fig.write_html(f"{plots_path}/pib_vs_consumo.html", auto_open=False)
print(f"✅ Gráfico interactivo guardado en: {plots_path}")

NameError: name 'os' is not defined

/data/gold/
 - ├── global_trend_per_beverage.parquet     ← Tendencias globales por bebida  
 - ├── region_trends.parquet                 ← Promedios por región  
 - ├── income_trends.parquet                 ← Promedios por grupo de ingreso  
 - ├── top10_countries.parquet               ← Ranking anual de países  
 - ├── plots/                                ← Gráficos HTML exportados  
 - └── pib_vs_consumo.html                   ← Scatter animado exportable

✅ Capa Gold completada: lista para Power BI o publicación en GitHub Pages.
